In [19]:
import torch
import torch.nn.functional as F
import numpy as np
import Preprocessor as pp
import sounddevice as sd # 마이크 입력을 위한 라이브러리

In [20]:
sd.default.device = (1, None)

In [21]:
SAVE_BEST_PATH = "./result/resnet152_best.pt"
CLASS_NAME = ["danger", "fire", "gas", "non", "tsunami"]
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 하이퍼 파라미터
CLASS_NUM = len(CLASS_NAME)
SAMPLE_RATE = 16000
DURATION = 1
NUM_SAMPLES = SAMPLE_RATE * DURATION
BATCH_SIZE = 32
LEARNING_RATE = 1e-4

In [22]:
def logmel_transform(audio_np):
    # numpy 배열을 torch 텐서로 변환, 배치 차원 추가: (1, num_samples)
    waveform = torch.tensor(audio_np, dtype=torch.float32)

    # 2채널 이상일때 1채널로 변환
    if waveform.shape[0] > 1:
        waveform = waveform.mean(axis=-1)
    
    # 오디오 샘플의 길이조정
    if waveform.shape[1] < NUM_SAMPLES: # 길이가 부족하면 0(무음)을 채워 길이를 연장
        waveform = F.pad(waveform, (0, NUM_SAMPLES - waveform.shape[1])) # num_samples와 현재의 길이의 차 만큼 0을 패딩
    else:
        waveform = waveform[:, :NUM_SAMPLES] # 길이가 길면 슬라이싱

    # 절댓값 정규화(-1 ~ 1)
    waveform = waveform / (waveform.abs().max() + 1e-9)

    logmel = pp.logmel(waveform).unsqueeze(0) # [batch, channel, mel_bins, time]

    return logmel


def predict(model, logmel, device=DEVICE):
    logmel = logmel.to(device)

    with torch.no_grad():
        outputs = model(logmel)
        probabilities = F.softmax(outputs, dim=1).cpu().numpy()[0]
        idx = probabilities.argmax()

    return idx, probabilities

In [23]:
model = torch.load(SAVE_BEST_PATH, map_location=DEVICE, weights_only=False)
model.to(DEVICE)
model.eval()
print()

In [24]:
print("실시간 마이크 입력을 시작합니다.")

try:
    # 실시간 마이크 입력 루프
    while True:
        # 오디오를 실시간으로 마이크에서 녹음
        audio_np = sd.rec(frames=NUM_SAMPLES, # 녹음할 샘플 수 (1초 분량)
                          samplerate=SAMPLE_RATE, # 샘플링 레이트
                          channels=1, # 단일 채널 (모노)
                          dtype="float32" # float32 타입으로 녹음
                          ).transpose(1, 0)
        sd.wait() # 녹음 완료까지 대기

        # audio_np = logmel(audio_np)
        logmel = logmel_transform(audio_np)
        idx, probabilities = predict(model, logmel)

        print(f"{CLASS_NAME[idx]} | {probabilities[idx] * 100:.2f}%")

except KeyboardInterrupt as e:
    print("\n실시간 마이크 입력을 종료합니다.\n", e)

실시간 마이크 입력을 시작합니다.
non | 92.23%
non | 93.97%
non | 99.08%
non | 79.96%
non | 75.29%
non | 94.86%
non | 93.36%
non | 87.91%
non | 91.72%
non | 85.79%
non | 81.98%
non | 83.58%
non | 87.44%
non | 96.74%
non | 84.82%
non | 91.58%
non | 93.49%
non | 98.64%
non | 93.13%
non | 84.00%
non | 97.17%
non | 91.38%
non | 80.60%
non | 89.18%
non | 98.01%
non | 74.33%
non | 97.15%
non | 94.70%
non | 82.29%
fire | 80.30%
non | 62.80%
fire | 84.22%
non | 65.48%
fire | 66.77%
non | 55.02%
non | 96.80%
non | 88.99%
non | 94.69%
non | 88.06%
non | 91.99%
non | 97.74%
non | 86.27%
fire | 50.96%
non | 81.10%
non | 94.11%
non | 82.55%
non | 94.31%
non | 92.93%
non | 93.23%
non | 97.62%
non | 90.38%
non | 95.88%
non | 91.44%
non | 95.56%
non | 92.35%
non | 88.21%
non | 77.75%
non | 92.94%
non | 86.32%
non | 80.44%
non | 70.04%
non | 76.58%
non | 60.66%
non | 93.58%
non | 53.44%
non | 72.57%
fire | 67.46%
non | 93.67%
non | 86.08%
non | 99.88%
non | 96.71%

실시간 마이크 입력을 종료합니다.
 
